## Ajuste Bouger y término de color

<pre>Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica
Jaime Zamorano and Nicolás Cardiel

Datos de campaña JKT 1999 Tesis Pablo Pérez González

v1  2021/12/11  J. Zamorano almost ready for 2020/21 
</pre>

In [ ]:
#!/usr/bin/env python
import matplotlib
from pylab import *
import numpy as np
from matplotlib.pyplot import show
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import integrate,interpolate,stats
import matplotlib.mlab as ml
from math import pi  
from matplotlib.pyplot import figure, show, rc, grid, savefig
from astropy.io import ascii

In [ ]:
mpl.rcParams['text.usetex']=False
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Verdana']
rcParams['font.size'] = 15.
rcParams['lines.linewidth'] = 4.
rcParams['axes.labelsize'] = 'large'
rcParams['grid.linewidth'] = 1.0
rcParams['grid.linestyle'] = ':'

### Datos observacionales 
Son observaciones con el telescopio JKT (La Palma) pertenecientea a la tesis de Pablo Pérez González obtenidos en Julio de 1999. Usaremos sus ficheros que contienen las medidas de las estrellas estándar. 

Leemos el fichero csv ('comma separated values') correspondiente a la primera noche de observación como una tabla de astropy. 

Nota: El flujo f en la columna 2.5logf está en unidades de cuentas/s 

In [ ]:
directory = "./files/fotometria_1/"
file_N1 = directory+"JKT99_N1_refer_stars_b.csv"
data1 = ascii.read(file_N1, format='basic', delimiter=';', guess=False)

In [ ]:
print(data1)    # cada columna es una variable

In [ ]:
# podemos referirnos a ella por su nombre que es el que aparece en el encabezado
print(data1['secz'][0:4])  

### Preparando la gráfica para ajustar la Recta de Bouguer

La ecuación simple de la fotometría
\begin{equation*}
m_B + 2.5 log(F_B)  =  C - K_B X  
\end{equation*}

In [ ]:
data1['B'] = np.add(data1['V'],data1['B-V'])               # para la mgnitud B estándar
data1['B+2.5logF'] = np.add(data1['B'],data1['2.5logf'])   # para el eje Y de la gráfica

In [ ]:
print(data1[0:6])

In [ ]:
figura = plt.figure(figsize=(10,6))
ax0 = plt.subplot(1,1,1)
xx = np.linspace(0.0,5,30)
plt.plot(data1['secz'],data1['B+2.5logF'],'go')
plt.xlim(0.0,3.0)
plt.ylim(21.5,23.0)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s)')
plt.title('Gráfica para la noche 1')


Se aprecia que los puntos se alinean como era de esperar.
Podemos ajustar una recta.

### Ajuste de la recta de Bouguer

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(data1['secz'],data1['B+2.5logF'])
print("slope, intercept, r_value, p_value, std_err")
print("%.3f  %.3f %.3f  %.3f %.3f" % (slope, intercept, r_value, p_value, std_err))

In [ ]:
(m_x1,b_x1) = polyfit(data1['secz'],data1['B+2.5logF'],1)
xx = np.linspace(0.0,5,30)
ynew = polyval([m_x1,b_x1],xx)
print("%.3f  %.3f" % (m_x1,b_x1))

La pendiente es negativa. El coeficiente de extinción $K_B$ = 0.461 es positivo.

In [ ]:
figura = plt.figure(figsize=(10,6))
ax0 = plt.subplot(1,1,1)
plt.plot(data1['secz'],data1['B+2.5logF'],'go',label='night 1')
plt.xlim(0.0,3.0)
plt.ylim(21.0,23.5)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s)')
plt.plot(xx,ynew,'k--',lw=2)
plt.legend()
ax0.text(0.05, 0.05,"B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x1,m_x1), ha='left', va='center', transform=ax0.transAxes,fontsize=15,bbox=dict(facecolor='white'))

Hacemos la misma operación para otras noches de la misma campaña

In [ ]:
file_N2 = directory+"JKT99_N2_refer_stars_b.csv"
data2 = ascii.read(file_N2, format='basic', delimiter=';', guess=False)
file_N6 = directory+"JKT99_N6_refer_stars_b.csv"
data6 = ascii.read(file_N6, format='basic', delimiter=';', guess=False)
file_N7 = directory+"JKT99_N7_refer_stars_b.csv"
data7 = ascii.read(file_N7, format='basic', delimiter=';', guess=False)

In [ ]:
data2['B']         = np.add(data2['V'],data2['B-V']) 
data2['B+2.5logF'] = np.add(data2['B'],data2['2.5logf']) 
data6['B']         = np.add(data6['V'],data6['B-V']) 
data6['B+2.5logF'] = np.add(data6['B'],data6['2.5logf']) 
data7['B']         = np.add(data7['V'],data7['B-V']) 
data7['B+2.5logF'] = np.add(data7['B'],data7['2.5logf']) 

In [ ]:
print('Night 1',"%.3f  %.3f" % (m_x1,b_x1))

(m_x2,b_x2) = polyfit(data2['secz'],data2['B+2.5logF'],1)
ynew2 = polyval([m_x2,b_x2],xx)
print('Night 2',"%.3f  %.3f" % (m_x2,b_x2))

x = [x for x,y in zip(data6['secz'],data6['B+2.5logF']) if x<1.8]
y = [y for x,y in zip(data6['secz'],data6['B+2.5logF']) if x<1.8]
#(m_x6,b_x6) = polyfit(data6['secz'],data6['B+2.5logF'],1)
(m_x6,b_x6) = polyfit(x,y,1)
xx = np.linspace(0.0,5,30)
ynew6 = polyval([m_x6,b_x6],xx)
print('Night 6',"%.3f  %.3f" % (m_x6,b_x6))
(m_x7,b_x7) = polyfit(data7['secz'],data7['B+2.5logF'],1)
ynew7 = polyval([m_x7,b_x7],xx)
print('Night 7',"%.3f  %.3f" % (m_x7,b_x7))

In [ ]:
figura = plt.figure(figsize=(10,6))
ax0 = plt.subplot(1,1,1)
plt.plot(data1['secz'],data1['B+2.5logF'],'go',label='night 1',ms=5)
plt.plot(data2['secz'],data2['B+2.5logF'],'ko',label='night 2',ms=4)
plt.plot(data6['secz'],data6['B+2.5logF'],'ro',label='night 6',ms=4)
plt.plot(data7['secz'],data7['B+2.5logF'],'bo',label='night 7',ms=5)
plt.plot(xx,ynew,'g--',lw=2)
plt.plot(xx,ynew6,'r--',lw=2)
plt.plot(xx,ynew7,'b--',lw=2)
plt.xlim(1.0,2.5)
plt.ylim(21.0,23.5)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s)')

plt.legend()
ax0.text(0.05, 0.24,"Night 1   B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x1,m_x1), ha='left', va='center', transform=ax0.transAxes,fontsize=12)
ax0.text(0.05, 0.18,"Night 2   B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x2,m_x2), ha='left', va='center', transform=ax0.transAxes,fontsize=12)
ax0.text(0.05, 0.11,"Night 6   B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x6,m_x6), ha='left', va='center', transform=ax0.transAxes,fontsize=12)
ax0.text(0.05, 0.05,"Night 7   B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x7,m_x7), ha='left', va='center', transform=ax0.transAxes,fontsize=12)

Esperamos cambios en el coeficiente de extinción pero no en la constante instrumental.
Para nuestra sorpresa la constante instrumental varía de una noche a otra.

### Término de color
Podemos pintar la gráfica de la noche 7 (por ejemplo) codificando el color de las estrellas para comprobar que los puntos de las estrellas más rojas (mayor índice de color B-V) se separan hacia arriba de la recta ajustada y viceversa. 

In [ ]:
figura = plt.figure(figsize=(10,6))
ax0 = plt.subplot(1,1,1)
plt.scatter(data7['secz'],data7['B+2.5logF'],c=data7['B-V'],label='night 7',edgecolor='black')
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 35
cbar.ax.set_ylabel('(B-V)', rotation=0)
plt.plot(xx,ynew7,'b--',lw=2)
plt.xlim(1.0,1.8)
plt.ylim(22.2,23.0)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s)')
plt.legend()
ax0.text(0.05, 0.05,"Night 7   B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x7,m_x7), ha='left', va='center', transform=ax0.transAxes,fontsize=12)


### Ajuste de un término de color

Intentamos ajustar un término de color en la forma
\begin{equation*}
m_B + 2.5 log(F_B)  =  C - K_B X + K_{B-R} (B-R) 
\end{equation*}

In [ ]:
data7['B-R'] = np.add(data7['B-V'],data7['V-R'])    # para calcular el B-R de las estrellas 

In [ ]:
figura = plt.figure(figsize=(10,6))
ax0 = plt.subplot(1,1,1)
plt.scatter(data7['secz'],data7['B+2.5logF'],c=data7['B-R'],label='night 7',edgecolor='black')
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 35
cbar.ax.set_ylabel('(B-R)', rotation=0)
plt.plot(xx,ynew7,'b--',lw=2)
plt.xlim(1.0,1.8)
plt.ylim(22.2,23.0)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s)')
plt.legend()
ax0.text(0.05, 0.05,"Night 7   B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x7,m_x7), ha='left', va='center', transform=ax0.transAxes,fontsize=12)

Definimos una función para realizar el ajuste
\begin{equation*}
m_B + 2.5 log(F_B)  =  C - K_B X + K_{B-R} (B-R) 
\end{equation*}

In [ ]:
def mifuncion(X , c, k, kbr):
    x,br = X
    return c - k * x + kbr * br

In [ ]:
from scipy.optimize import curve_fit
# ajustamos la función
x  = data7['secz']
br = data7['B-R']
y  = data7['B+2.5logF']
# estimaciones de partida C,k,kbr:
p0 = 22., 0.4, 0.01
popt_N7, _ = curve_fit(mifuncion,(x,br),y,p0)

In [ ]:
print(popt_N7)     # Cb Kb Kbr

In [ ]:
print('Night 7',"%.3f  %.3f" % (b_x7,m_x7))
print('Night 7',"%.3f  %.3f %.3f" % (popt_N7[0],popt_N7[1],popt_N7[2]))
termino = np.multiply(popt_N7[2],data7['B-R'])
ynew = np.subtract(data7['B+2.5logF'],termino)

In [ ]:
figura = plt.figure(figsize=(12,6))
ax0 = plt.subplot(121)
plt.scatter(x,y,c=data7['B-R'],label='night 7',edgecolor='black')
plt.xlim(1.0,1.8)
plt.ylim(22.2,23.0)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s)')
plt.legend()
ax0.text(0.05, 0.05,"B + 2.5 log F = %.3f  %.3f sec(z) "%(b_x7,m_x7), ha='left', va='center', transform=ax0.transAxes,fontsize=12)

ax1 = plt.subplot(122)
plt.plot(x,ynew,'bs')
plt.xlim(1.0,1.8)
plt.ylim(22.2,23.0)
plt.grid()
plt.xlabel('sec(z)')
plt.ylabel('B + 2.5 log F (c/s) + Kbr (B-R)')
plt.legend()
ax1.text(0.05, 0.05,"B + 2.5 log F = %.3f - %.3f sec(z) + %.3f (B-R)"%(popt_N7[0],popt_N7[1],popt_N7[2]), ha='left', va='center', transform=ax1.transAxes,fontsize=12)
plt.tight_layout()

La dispersión de los puntos azules (los corregidos de término de color) es mucho menor.

In [ ]:
data6['B-R'] = np.add(data6['B-V'],data6['V-R'])    # para calcular el B-R de las estrellas 
x  = data6['secz']
br = data6['B-R']
y  = data6['B+2.5logF']
# estimaciones de partida C,k,kbr:
p0 = 22., 0.4, 0.01
popt_N6, _ = curve_fit(mifuncion,(x,br),y,p0)
print(popt_N6)     # Cb Kb Kbr

In [ ]:
data2['B-R'] = np.add(data2['B-V'],data2['V-R'])    # para calcular el B-R de las estrellas 
x  = data2['secz']
br = data2['B-R']
y  = data2['B+2.5logF']
# estimaciones de partida C,k,kbr:
p0 = 22., 0.4, 0.01
popt_N2, _ = curve_fit(mifuncion,(x,br),y,p0)
print(popt_N2)     # Cb Kb Kbr

In [ ]:
data1['B-R'] = np.add(data1['B-V'],data1['V-R'])    # para calcular el B-R de las estrellas 
x  = data1['secz']
br = data1['B-R']
y  = data1['B+2.5logF']
# estimaciones de partida C,k,kbr:
p0 = 22., 0.4, 0.01
popt_N1, _ = curve_fit(mifuncion,(x,br),y,p0)
print('Night  ',"C_B    K_B   K_{B-R}")
print('Night 1',"%.3f %.3f %.3f" % (popt_N1[0],popt_N1[1],popt_N1[2]))     # Cb Kb Kbr
print('Night 2',"%.3f %.3f %.3f" % (popt_N2[0],popt_N2[1],popt_N2[2]))     # Cb Kb Kbr
print('Night 6',"%.3f %.3f %.3f" % (popt_N6[0],popt_N6[1],popt_N6[2]))     # Cb Kb Kbr
print('Night 7',"%.3f %.3f %.3f" % (popt_N7[0],popt_N7[1],popt_N7[2]))     # Cb Kb Kbr